#
<div align=center>
<img src="https://uol.unifor.br/acesso/app/autenticacao/assets/img/logos/icon-unifor.svg" width=45 height=45>

<img src="https://vortex.unifor.br/assets/logos/v1.png" width=45 height=45>
<font size=5 color=k>
<br><br>
<font size=5 color=k><strong>Projeto:</strong> Bone Age

<strong>Etapa:</strong> Treinamento e teste

<strong>Autoria:</strong> Heitor Teixeira

</div>

## 
<font size=5 color='blue'> 1 - Bibliotecas e configuração de GPU

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import random
import GPUtil
from tensorflow.keras.optimizers import Adam, SGD, RMSprop # type: ignore
from tensorflow.keras.losses import MeanAbsoluteError, MeanSquaredError, Huber, LogCosh, MeanSquaredLogarithmicError # type: ignore
from tensorflow.keras.metrics import RootMeanSquaredError # type: ignore
from auxiliar.dataset import criar_dataset
from auxiliar.avaliar import carregar_pesos_e_avaliar_modelos
from auxiliar.callbacks import callbacks
from auxiliar.arquiteturas import custom_model

tf_gpu = tf.config.experimental.list_physical_devices('GPU')
if tf_gpu:
    try:
        nome_gpu = GPUtil.getGPUs()
        print("GPU configurada: ", nome_gpu[0].name)
        tf.config.experimental.set_memory_growth(tf_gpu[0], True)
    except RuntimeError as e:
        print(e)


GPU configurada:  NVIDIA GeForce RTX 4080
Physical devices cannot be modified after being initialized


In [2]:
id_rodada = 23
rede = 'EfficientNetV2L'

epocas = 100
batch_size = 8
loss = MeanAbsoluteError() 
nome_loss = 'mae' # uma string pra salvar no csv
lr = 0.0005

otimizador = Adam(learning_rate = lr) # Adam, SGD, RMSprop
nome_otimizador = 'sgd'
attention = '-' # 'se' ou 'cbam'. as 2 camadas de attention que adicionei caso queria treinar com
camada_pooling = 'avg' # 'avg' ou 'max' pra camada de pooling que tira a média ou o maior valor
dense_1 = 1024
dropout_1 = 0.3
dense_2 = 512
dropout_2 = 0.3
dense_3 = 128
dropout_3 = 0.1

data_aug = False

check_best = True
check_10 = True
early_stop = True
log = True
reduce_lr = True

dir_dados = '../treino-validacao-teste.csv'
dir_imagens = '../imagens/imagens'
teste_size = 0.3
val_size = 0.1

dir_modelos_salvos = f'./redes/{rede}/modelos_salvos/{rede}_{id_rodada}_epoca_{{epoch:02d}}.hdf5'
dir_csv_log_treino = f'./redes/{rede}/log_treino/{rede}_{id_rodada}_treinamento_log.csv'


## 
<font size=5 color='blue'> 2 - Importando dados e criando datasets

### 
<font size=3 color=green> 2.2 - Dados de treino e teste

In [3]:
dados_treino = pd.read_csv(dir_dados)

dados_treino, dados_teste = train_test_split(
    dados_treino, 
    test_size = teste_size,
)

dados_treino, dados_validacao = train_test_split(
    dados_treino, 
    test_size = val_size,
)

print(len(dados_treino))
print(len(dados_validacao))
print(len(dados_teste))

9716
1080
4627


In [4]:

dataset_treino = criar_dataset(
    dataframe = dados_treino,
    diretorio = dir_imagens,
    batch_size = batch_size,
    rede = rede,
    shuffle = True,
    data_aug = data_aug
)

dataset_validacao = criar_dataset(
    dataframe = dados_validacao,
    diretorio = dir_imagens,
    batch_size = batch_size,
    rede = rede,
)


dataset_teste = criar_dataset(
    dataframe = dados_teste,
    diretorio = dir_imagens,
    batch_size = batch_size,
    rede = rede,

)

## 
<font size=5 color='blue'> 3 - Modelando a rede

In [5]:
model = custom_model(
    rede= rede,
    loss = loss, 
    otimizador = otimizador, 
    attention = attention, 
    camada_densa_1 = dense_1,
    dropout_1 = dropout_1, 
    camada_densa_2 = dense_2,
    dropout_2 = dropout_2,
    camada_densa_3 = dense_3,
    dropout_3 = dropout_3,
    pooling = camada_pooling
    )

num_layers = len(model.layers)
print(f'camadas no modelo: {num_layers}')

model.summary()

modelo compilado
camadas no modelo: 422
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling (Rescaling)          (None, None, None,   0           ['input_1[0][0]']                
                                3)                                                                
                                                                                                  
 normalization (Normalization)  (None, None, None,   0           ['rescaling[0][0]']              
                                3)                    

In [6]:
'''
callbacks executadas durante o treinamento.
    1 - salva o melhor modelo de acordo com a melhor validcao da metrica
    2 - salva o modelo de 10 em 10 epocas
    3 - termina o treinamento se nao houver melhora em N epocas (nao deixei configuravel. porem, da pra mudar -> N = 10)
    4 - salva um csv de log
    5 - reduz automaticamente a lr se nao houver melhora em N epocas em 50% (da pra configurar, mas deixei nesses valores fixos)
'''

callbacks_treino = callbacks(
    dir_modelos_salvos = dir_modelos_salvos, 
    dir_csv_log = dir_csv_log_treino,
    check_best = check_best, 
    check_15 = check_10, 
    early_stop = early_stop, 
    log = log, 
    reduce_lr = reduce_lr,
    )

callbacks adicionados:
check best
check 15
early stop
log
reduzindo lr


## 
<font size=5 color='blue'> 4 - Treinar modelo

In [7]:
steps_per_epoch = len(dados_treino) // batch_size
validation_steps=len(dados_validacao) // batch_size

model.fit(
    dataset_treino, 
    validation_data = dataset_validacao, 
    steps_per_epoch = steps_per_epoch, 
    validation_steps = validation_steps, 
    epochs = epocas,
    callbacks=callbacks_treino
    )
    

Epoch 1/100
194/194 [==============================] - ETA: 0s - loss: 25.0670 - mean_absolute_error: 25.0670 - mean_squared_error: 1390.1403 - root_mean_squared_error: 37.2846
Epoch 1: val_loss improved from inf to 13.27782, saving model to ./redes/EfficientNetV2B3/modelos_salvos\EfficientNetV2B3_22_epoca_01.hdf5
194/194 [==============================] - 77s 288ms/step - loss: 25.0670 - mean_absolute_error: 25.0670 - mean_squared_error: 1390.1403 - root_mean_squared_error: 37.2846 - val_loss: 13.2778 - val_mean_absolute_error: 13.2778 - val_mean_squared_error: 271.5018 - val_root_mean_squared_error: 16.4773 - lr: 0.0010
Epoch 2/100
194/194 [==============================] - ETA: 0s - loss: 16.0900 - mean_absolute_error: 16.1196 - mean_squared_error: 423.6711 - root_mean_squared_error: 20.5493
Epoch 2: val_loss improved from 13.27782 to 9.93346, saving model to ./redes/EfficientNetV2B3/modelos_salvos\EfficientNetV2B3_22_epoca_02.hdf5
194/194 [==============================] - 54s 279m

## 
<font size=5 color='blue'> 5 - Avaliar modelo

In [8]:
carregar_pesos_e_avaliar_modelos(
    model=model,
    rede=rede,
    id_rodada=id_rodada,  
    dataset_teste=dataset_teste,
    dados_teste=dados_teste,
    batch_size=batch_size,
    imagenet_descongelada=True,
    data_aug=data_aug,
    nome_otimizador=nome_otimizador,
    nome_loss=nome_loss,
    lr=0.001,
    attention=attention,
    camada_pooling=camada_pooling,
    dense_1=dense_1,
    dropout_1=dropout_1,
    dense_2=dense_2,
    dropout_2=dropout_2,
    dense_3=dense_3,
    dropout_3=dropout_3
)

93/93 [==============================] - 7s 79ms/step - loss: 13.7851 - mean_absolute_error: 13.7973 - mean_squared_error: 302.9643 - root_mean_squared_error: 17.3746
modelo EfficientNetV2B3_22_epoca_01.hdf5 - resultado: [13.7850980758667, 13.79725456237793, 302.9642639160156, 17.374622344970703]
93/93 [==============================] - 7s 65ms/step
MAE Manual para o modelo EfficientNetV2B3_22_epoca_01.hdf5: 13.785093397050227
93/93 [==============================] - 6s 66ms/step - loss: 10.6261 - mean_absolute_error: 10.6422 - mean_squared_error: 198.6829 - root_mean_squared_error: 14.0764
modelo EfficientNetV2B3_22_epoca_02.hdf5 - resultado: [10.626089096069336, 10.642182350158691, 198.68292236328125, 14.07639217376709]
93/93 [==============================] - 6s 66ms/step
MAE Manual para o modelo EfficientNetV2B3_22_epoca_02.hdf5: 10.626089834564057
93/93 [==============================] - 6s 67ms/step - loss: 10.2901 - mean_absolute_error: 10.3026 - mean_squared_error: 171.8234 - r